In [551]:
import pandas as pd
import psycopg2 as postgres


In [552]:
BASE = 'tp_dba'
USER = 'axelmoriceau'
PSWD = ''

def connect():
    conn = postgres.connect(
        host='localhost',
        database=BASE,
        user=USER,
        password=PSWD,
        port='5432'
    )
    return conn
conn = connect()
cur = conn.cursor()

In [553]:
cur.execute(open("sql/questions/reset.sql", "r").read())

In [554]:
# requête interactive
region = 'aquitaine' # mettre a jour le contenu ici pour changer de région (i.e: tagne -> Bretagne, compte -> Franche compté etc)

cur.execute("select d.libelle from departements d join regions r using(reg) where r.ncc LIKE UPPER(%(region)s) or r.libelle LIKE UPPER(%(region)s) order by d.libelle", {'region': f'%{region}%'})
conn.commit()
data = cur.fetchall()

print(f'Les départements qui correspondent a la région {region}:')
for dep in data:
    print(f'\t- {dep[0]}')


Les départements qui correspondent a la région aquitaine:
	- Charente
	- Charente-Maritime
	- Corrèze
	- Creuse
	- Deux-Sèvres
	- Dordogne
	- Gironde
	- Haute-Vienne
	- Landes
	- Lot-et-Garonne
	- Pyrénées-Atlantiques
	- Vienne


In [555]:
commune = 'bordeaux'  # le nom de la ville doit être exact, sans accent ni caractère spéciaux
indicateur = 'pop'  # parmis DECE, LOG, LOGVAC, NAIS, NPER, PMEN, POP, RP, RSECOCC, SUPERF

cur.execute("select annee, ncc, dep,valeur from communes c join statistiques s on s.codgeo = c.com where upper(s.indicateur) = upper(%(indicateur)s) and c.ncc = upper( %(commune)s)", 
            {'commune': commune, 'indicateur': indicateur})
conn.commit()
data = cur.fetchall()

if(len(data)):
    print(f"Information pour l'indicateur {indicateur} pour la commune {commune} ({data[0][2]})")
    for info in data:
        print(f'\t{info[0]}: {info[3]:,}')
else:
    print(f'aucune information pour la commune de {commune} et l\'indicateur {indicateur}')

Information pour l'indicateur pop pour la commune bordeaux (33 )
	1968: 257,070
	1975: 257,070
	1982: 257,070
	1990: 257,070
	1999: 257,070
	2008: 257,070
	2013: 257,070
	2018: 257,871


In [556]:
# Question 1
cur.execute(open("sql/questions/question_1/question_1#1.sql", "r").read())
conn.commit()
data = cur.fetchall()

print('Les 10 villes les plus peuplées de France en 2018:')
for ville in data:
    print(f'\t- {ville[0]}: {ville[1]:,} habitants')


Les 10 villes les plus peuplées de France en 2018:
	- Paris: 2,175,702 habitants
	- Marseille: 868,277 habitants
	- Lyon: 518,635 habitants
	- Toulouse: 486,828 habitants
	- Nice: 341,032 habitants
	- Montpellier: 290,053 habitants
	- Strasbourg: 284,677 habitants
	- Bordeaux: 257,871 habitants
	- Lille: 233,098 habitants
	- Rennes: 217,728 habitants


In [557]:
# Question 2
cur.execute(open("sql/questions/question_1/question_1#2.sql", "r").read())
conn.commit()
data = cur.fetchall()

print('Évolution de la population Française au fil des années:')
for population in data:
    population = [0 if v is None else v for v in population]
    print(f'\n\t- Année {population[0]}: {population[1]:,} habitants -> +{population[2]:,} habitants ({population[3]}%)')
    print(f'\t\t- Naissances: {population[4]:,} | Décès: {population[5]:,}')
    if(population[2]):
        print(f'\t\t- Augmentation: {population[2]:,} || Naissances - Décès: {population[4] - population[5]:,} || Différence: {population[2]-(population[4] - population[5]):,}')


Évolution de la population Française au fil des années:

	- Année 1968: 50,788,520 habitants -> +0 habitants (0%)
		- Naissances: 6,378,063 | Décès: 4,144,525

	- Année 1975: 53,798,003 habitants -> +3,009,483 habitants (5.5940422175150255%)
		- Naissances: 5,767,687 | Décès: 4,149,276
		- Augmentation: 3,009,483 || Naissances - Décès: 1,618,411 || Différence: 1,391,072

	- Année 1982: 55,618,453 habitants -> +1,820,450 habitants (3.273104341826983%)
		- Naissances: 6,599,046 | Décès: 4,613,309
		- Augmentation: 1,820,450 || Naissances - Décès: 1,985,737 || Différence: -165,287

	- Année 1990: 58,087,393 habitants -> +2,468,940 habitants (4.250388720320087%)
		- Naissances: 7,110,433 | Décès: 5,057,784
		- Augmentation: 2,468,940 || Naissances - Décès: 2,052,649 || Différence: 416,291

	- Année 1999: 60,191,608 habitants -> +2,104,215 habitants (3.495861084156449%)
		- Naissances: 7,442,930 | Décès: 5,074,193
		- Augmentation: 2,104,215 || Naissances - Décès: 2,368,737 || Différence: -

In [558]:
# Question 3
cur.execute(open("sql/questions/question_1/question_1#3.sql", "r").read())
conn.commit()
data = cur.fetchall()

print('Ville la plus peuplée de chaque région en 2018:')
for population in data:
    print(f'\t- Région {population[2]} -> {population[1]} ({population[0]}): {population[3]:,} habitants')


Ville la plus peuplée de chaque région en 2018:
	- Région Île-de-France -> Paris (75056): 2,175,702 habitants
	- Région Provence-Alpes-Côte d'Azur -> Marseille (13055): 868,277 habitants
	- Région Auvergne-Rhône-Alpes -> Lyon (69123): 518,635 habitants
	- Région Occitanie -> Toulouse (31555): 486,828 habitants
	- Région Grand Est -> Strasbourg (67482): 284,677 habitants
	- Région Nouvelle-Aquitaine -> Bordeaux (33063): 257,871 habitants
	- Région Hauts-de-France -> Lille (59350): 233,098 habitants
	- Région Bretagne -> Rennes (35238): 217,728 habitants
	- Région Normandie -> Le Havre (76351): 169,733 habitants
	- Région Bourgogne-Franche-Comté -> Dijon (21231): 156,854 habitants
	- Région Pays de la Loire -> Angers (49007): 154,508 habitants
	- Région La Réunion -> Saint-Denis (97411): 150,535 habitants
	- Région Centre-Val de Loire -> Tours (37261): 136,463 habitants
	- Région Martinique -> Fort-de-France (97209): 78,126 habitants
	- Région Corse -> Ajaccio (2A004): 70,817 habitants
	

In [559]:
# Création des vues

cur.execute(open("sql/questions/question_2/question_2#vue_region.sql", "r").read())
cur.execute(open("sql/questions/question_2/question_2#vue_departements.sql", "r").read())
conn.commit()


In [560]:
# Ajout des colonnes population
cur.execute(open("sql/questions/question_3/alter_tables.sql", "r").read())
cur.execute("select ncc,population from regions order by ncc limit 5;")
conn.commit()

data = cur.fetchall()
for region in data:
    print(f'{region[0]}: {region[1]}')


AUVERGNE RHONE ALPES: None
BOURGOGNE FRANCHE COMTE: None
BRETAGNE: None
CENTRE VAL DE LOIRE: None
CORSE: None


In [561]:

# Creation et appel de la procédure
cur.execute(open("sql/questions/question_3/procedure.sql", "r").read())
cur.execute("call proc_update_population()")
cur.execute("select ncc,population from regions order by ncc limit 5;")
conn.commit()

data = cur.fetchall()
for region in data:
    print(f'{region[0]}: {region[1]}')


AUVERGNE RHONE ALPES: 7994459
BOURGOGNE FRANCHE COMTE: 2807807
BRETAGNE: 3335226
CENTRE VAL DE LOIRE: 2572853
CORSE: 338554


# Note
Les 2 blocs suivants sont executés dans l'ordre inverse de leur appartition sur l'ennoncé du TP, nous n'avons pas réussi a rendre opérationnel la mise a jour automatiques des populations si les tables régions et départements rejettent toute mise a jour/insertion/suppression.

On aurait pu bloquer les tables au niveau des droits et permettre a la procédure de s'executer mais la question portant sur les trigger nous avons fait le blocage par trigger.

Ainsi pour tester elles s'executent dans l'ordre inverse à la demande:
1. création d'un trigger sur la table de statistiques qui ne se déclenche qu'à la suppression ou si la mise à jour/insertion implique l'indicateur de population
2. mise a jour de la tables statistiques pour augmenter une population (cf: cellule suivante)
3. création des triggers de blocage d'opération
4. test de mise a jour des tables régions et départements afin de constater que l'erreur que nous renvoyons depuis le trigger nous ai bien retournée

In [562]:
# Mise a jour auto des tables régions et départements quand un indicateur de population est modifié

ville = 'bordeaux'  # ville ciblée (se base sur le ncc)
augmentation = 1 # permet de définir de combien l'on augmente la population de la ville donnée

# ne pas modifier en dessous
sql = """
    select r.libelle,r.population, d.libelle,d.population, c.libelle,valeur::integer, c.com
    from communes c 
    join statistiques s on s.codgeo = c.com 
    join departements d using(dep) 
    join regions r using(reg) 
    where c.ncc = upper(%(ville)s) 
    and s.annee = 2018 
    and s.indicateur = 'POP'
    group by valeur, r.population, d.population, r.libelle, c.libelle, d.libelle, c.com;
""" # ne pas modifier


# dictionnaires utilisés pour constater les différences
pre_update = {'region' : None, 'dep': None, 'ville': None} 
post_update = {'region': None, 'dep': None, 'ville': None}

# ajout du trigger
cur.execute(open("sql/questions/question_3/triggers_2.sql", "r").read())
cur.execute(sql, {'ville': ville})
conn.commit()
pre_update = list(cur.fetchone())
try:
    cur.execute("update statistiques set valeur = %(valeur)s where indicateur = 'POP' and annee = 2018 and codgeo = %(codgeo)s;", {'valeur': pre_update[-2]+augmentation, 'codgeo': str(pre_update[-1])})
    cur.execute(sql, {'ville': ville})
    conn.commit()
    post_update = list(cur.fetchone())
    print("Avant mise à jour:")
    print(f'Région: {pre_update[0]} -> {pre_update[1]} habitants')
    print(f'Département: {pre_update[2]} -> {pre_update[3]} habitants')
    print(f'Ville: {pre_update[4]} -> {pre_update[5]} habitants')
    print("\nAprès mise à jour:")
    print(f'Région: {post_update[0]} -> {post_update[1]} habitants')
    print(f'Département: {post_update[2]} -> {post_update[3]} habitants')
    print(f'Ville: {post_update[4]} -> {post_update[5]} habitants')
except Exception as error:
    print(error)


Avant mise à jour:
Région: Nouvelle-Aquitaine -> 5980581 habitants
Département: Gironde -> 1602648 habitants
Ville: Bordeaux -> 257871 habitants

Après mise à jour:
Région: Nouvelle-Aquitaine -> 5980582 habitants
Département: Gironde -> 1602649 habitants
Ville: Bordeaux -> 257872 habitants


In [563]:
# Blocage des tables régions et départements

cur.execute(open("sql/questions/question_3/triggers_1.sql", "r").read())
conn.commit()
try:
    cur.execute("update regions set population = 10000 where reg = '75';")
except Exception as error:
    print(error)  # UPDATE on regions is not allowed.
try:
    conn = connect()
    cur = conn.cursor()
    cur.execute("update departements set population = 10000 where dep = '33';")
except Exception as error:
    print(error)  # UPDATE on departements is not allowed.


UPDATE on regions is not allowed.
CONTEXT:  PL/pgSQL function fun_prevent() line 3 at RAISE

UPDATE on departements is not allowed.
CONTEXT:  PL/pgSQL function fun_prevent() line 3 at RAISE



In [564]:
# conn.close()